# Deploy Agent To Vertex AI Agent Engine

After building and testing an agent built with ADK the next step is deployment for use in production workflows and applications.  The notebook workflow shows the steps for deploying the agent built in the product on [Vertex AI Agent Engine](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview).  This service includes:
- [Development](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/develop/adk) tooling to test the agent locally first
- [Deployment](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/deploy) tooling to create the agent instance on Vertex AI Agent Engine
- [Agent Use](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/use/adk) SDK directly within the Vetex AI SDK
- [Mangagement](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/manage/overview) services for access control, tracing, logging, and monitoring of deployed agents
- and services like [Agent Evaluation](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/evaluate) and more!

While Vertex AI Agent Engine is a service for many agent building frameworks (ADK, LangChain, LangGraph, AG2, LlamaIndex, Custom, ...) it is also directly integrated with Google ADK, the framework used in this project.  [ADK Deployment](https://google.github.io/adk-docs/deploy/) options include [Vertex AI Agent Engine](https://google.github.io/adk-docs/deploy/agent-engine/) as well as [Cloud Run](https://google.github.io/adk-docs/deploy/cloud-run/) and [custom infrastructure](https://google.github.io/adk-docs/deploy/gke/) deploymetns.

**Notes:**
- This notebook expects the current working directory to contain the agents folder. In other words, `agent.py` is at `./document_agent/agent.py`.

Package Imports:

In [31]:
import os, sys

import dotenv
#from dotenv import load_dotenv

from vertexai.preview import reasoning_engines
from vertexai import agent_engines
import vertexai

Verify working directory:

In [32]:
os.getcwd()

'/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/document_agent'

Review the local files, expecting the `document_agent` folder to be here:

In [33]:
os.listdir('.')

['document_agent', 'apps', 'deploy-vertex-ai-agent-engine.ipynb']

Verify the current Python version and executable location (the `venv` setup by this project):

In [34]:
sys.version, sys.executable

('3.13.3 (main, Apr 14 2025, 14:06:24) [GCC 12.2.0]',
 '/home/statmike/repos/vertex-ai-mlops/Applied ML/Solution Prototypes/document-processing/7-agents/.venv/bin/python')

Setup Clients:

In [36]:
# Load environment variables set with agent (./document_agent/.env)
dotenv.load_dotenv(dotenv_path = os.path.join('document_agent', '.env'))

# Vertex AI Initializaiton:
vertexai.init(
    project = os.getenv('GOOGLE_CLOUD_PROJECT'),
    location = os.getenv('GOOGLE_CLOUD_LOCATION'),
    staging_bucket = f"gs://{os.getenv('GOOGLE_CLOUD_STORAGE_BUCKET')}"
)

---
## Local Testing: Prepare Agent for Vertex AI Agent Engine

Before deploying the ADK agent to Vertex AI Agent Engine it can first be tested locally with the SDK.

In [6]:
from document_agent.agent import root_agent

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [7]:
session = app.create_session(user_id = 'u_123')
session

Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749316598.6712692)

In [8]:
app.list_sessions(user_id = 'u_123')

ListSessionsResponse(sessions=[Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749316598.6712692)])

In [9]:
session = app.get_session(user_id="u_123", session_id=session.id)
session

Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[], last_update_time=1749316598.6712692)

In [11]:
for event in app.stream_query(
    user_id = 'u_123',
    session_id = session.id,
    message = 'What do you do?'
):
    print(event)

{'content': {'parts': [{'text': 'I am the primary agent that processes document related requests. I can load documents, extract information, classify them, and compare them to vendor templates. To start, please upload a PDF/PNG file or provide the GCS URI (bucket and path) for the document you want to process.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 59, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 59}], 'prompt_token_count': 3474, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3474}], 'total_token_count': 3533, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-b6ed94f8-4582-41ef-9d39-c306a1ca024d', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'sDT5rKML', 'timestamp': 1749316602.635473}


In [12]:
for event in app.stream_query(
    user_id = 'u_123',
    session_id = session.id,
    message = 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-7c70745d-1670-473b-82c8-fd234e8ddc92', 'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf', 'gcs_bucket': 'statmike-mlops-349915'}, 'name': 'get_gcs_file'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 56, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 56}], 'prompt_token_count': 3583, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3583}], 'total_token_count': 3639, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-989f09b7-7f2b-4bf8-afa6-8526eb7ce9cb', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': set(), 'id': 'F7Y2oybx', 'timestamp': 1749316611.116951}
{'content': {'parts': [{'function_response': {'id': 'adk-7c70745d-1670-473b-82c8-fd234e8ddc92', 'n

In [13]:
event

{'content': {'parts': [{'text': 'Okay, I have loaded the file from GCS. The artifact key for this document is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`.\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 68,
  'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 68}],
  'prompt_token_count': 3751,
  'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>,
    'token_count': 3751}],
  'total_token_count': 3819,
  'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>},
 'invocation_id': 'e-989f09b7-7f2b-4bf8-afa6-8526eb7ce9cb',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': 'aT0ueQYB',
 'timestamp': 1749316614.02341}

In [14]:
app.get_session(user_id = 'u_123', session_id = session.id)

Session(id='cc428561-8e3c-4e01-829c-33924d731ca7', app_name='default-app-name', user_id='u_123', state={}, events=[Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='What do you do?')], role='user'), grounding_metadata=None, partial=None, turn_complete=None, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=None, invocation_id='e-b6ed94f8-4582-41ef-9d39-c306a1ca024d', author='user', actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}), long_running_tool_ids=None, branch=None, id='bzp9rqYP', timestamp=1749316602.632191), Event(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executa

In [15]:
for event in app.get_session(user_id = 'u_123', session_id = session.id).events:
    print(event)

content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='What do you do?')], role='user') grounding_metadata=None partial=None turn_complete=None error_code=None error_message=None interrupted=None custom_metadata=None usage_metadata=None invocation_id='e-b6ed94f8-4582-41ef-9d39-c306a1ca024d' author='user' actions=EventActions(skip_summarization=None, state_delta={}, artifact_delta={}, transfer_to_agent=None, escalate=None, requested_auth_configs={}) long_running_tool_ids=None branch=None id='bzp9rqYP' timestamp=1749316602.632191
content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, file_data=None, thought_signature=None, code_execution_result=None, executable_code=None, function_call=None, function_response=None, text='I am the primary agent that processes document related requests. I can load doc

In [16]:
app.delete_session(user_id = 'u_123', session_id = session.id)

In [17]:
app.list_sessions(user_id = 'u_123')

ListSessionsResponse(sessions=[])

more
- create session with specified id

---
## Deploy To Vertex AI Agent Engine

Now that the ADK Agent has been tested locally it can be deployed to Vertex AI Agent Engine.

In [18]:
remote_app = agent_engines.create(
    agent_engine = root_agent,
    requirements = "../requirements.txt",
    extra_packages = ["./document_agent"],
    gcs_dir_name = 'applied-ml-solution-prototypes/document-processing/agent_engine/staging',
    display_name = root_agent.name,
    description = root_agent.description,
    #env_vars = 
)

Deploying google.adk.agents.Agent as an application.
Reading requirements from requirements='../requirements.txt'
Read the following lines: ['absl-py==2.3.0', 'aiofiles==24.1.0', 'altair==5.5.0', 'annotated-types==0.7.0', 'anyio==4.9.0', 'asttokens==3.0.0', 'attrs==25.3.0', 'audioop-lts==0.2.1', 'Authlib==1.5.2', 'blinker==1.9.0', 'cachetools==5.5.2', 'certifi==2025.4.26', 'cffi==1.17.1', 'charset-normalizer==3.4.1', 'click==8.1.8', 'cloudpickle==3.1.1', 'comm==0.2.2', 'cryptography==44.0.2', 'db-dtypes==1.4.3', 'debugpy==1.8.14', 'decorator==5.2.1', 'deepdiff==6.7.1', 'Deprecated==1.2.18', 'docstring_parser==0.16', 'executing==2.2.0', 'fastapi==0.115.12', 'ffmpy==0.5.0', 'filelock==3.18.0', 'Flask==3.1.1', 'fsspec==2025.5.1', 'gitdb==4.0.12', 'GitPython==3.1.44', 'google-adk==1.2.1', 'google-api-core==2.24.2', 'google-api-python-client==2.169.0', 'google-auth==2.39.0', 'google-auth-httplib2==0.2.0', 'google-cloud-aiplatform==1.96.0', 'google-cloud-appengine-logging==1.6.1', 'google-cl

In [19]:
remote_app.resource_name

'projects/1026793852137/locations/us-central1/reasoningEngines/1988805428414251008'

In [37]:
dotenv.set_key(
    './apps/.env', 'ADK_DEPLOY_RESOURCE_ID', remote_app.resource_name
)

(True,
 'ADK_DEPLOY_RESOURCE_ID',
 'projects/1026793852137/locations/us-central1/reasoningEngines/1988805428414251008')

In [20]:
remote_session = remote_app.create_session(user_id = 'u_123')
remote_session

{'id': '4622554141112139776',
 'lastUpdateTime': 1749316979.885203,
 'events': [],
 'state': {},
 'appName': '1988805428414251008',
 'userId': 'u_123'}

In [21]:
remote_app.list_sessions(user_id = 'u_123')

{'sessions': [{'id': '4622554141112139776',
   'events': [],
   'lastUpdateTime': 1749316979.885203,
   'state': {},
   'appName': '1988805428414251008',
   'userId': 'u_123'}]}

In [22]:
remote_session = remote_app.get_session(user_id = "u_123", session_id = remote_session['id'])
remote_session

{'id': '4622554141112139776',
 'lastUpdateTime': 1749316979.885203,
 'state': {},
 'events': [],
 'appName': '1988805428414251008',
 'userId': 'u_123'}

In [23]:
for event in remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'What do you do?'
):
    print(event)

{'content': {'parts': [{'text': 'I am the primary agent for processing document-related requests. I can extract information from documents, classify them, and compare them to vendor templates. To get started, please upload a PDF or PNG file, or provide the GCS URI (bucket and path) of the document you want to process.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 61, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 61}], 'prompt_token_count': 3474, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3474}], 'total_token_count': 3535, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-189f42c5-c34d-4d2a-b44e-136233d8f2cf', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': '7S92HXO0', 'timestamp': 1749316982.735535}


In [24]:
for event in remote_app.stream_query(
    user_id = 'u_123',
    session_id = remote_session['id'],
    message = 'use this file: gs://statmike-mlops-349915/applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf'
):
    print(event)

{'content': {'parts': [{'function_call': {'id': 'adk-6580e54f-9b12-4204-94c4-a3a8b73d5229', 'args': {'gcs_file_path': 'applied-ml-solution-prototypes/document-processing/vendor_2/fake_invoices/vendor_2_invoice_10.pdf', 'gcs_bucket': 'statmike-mlops-349915'}, 'name': 'get_gcs_file'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 56, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 56}], 'prompt_token_count': 3585, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3585}], 'total_token_count': 3641, 'traffic_type': 'ON_DEMAND'}, 'invocation_id': 'e-38cbcc96-0ba3-4539-b83b-a05a136c4ef5', 'author': 'document_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids': [], 'id': 'RLdg5dbO', 'timestamp': 1749316984.305393}
{'content': {'parts': [{'function_response': {'id': 'adk-6580e54f-9b12-4204-94c4-a3a8b73d5229', 'name': 'get_gcs_file', 'response': {'result': 'The file vendor_2_invoice_

In [25]:
event

{'content': {'parts': [{'text': 'OK. I have loaded the file from GCS. The artifact key is `gcsfile_statmike-mlops-349915_applied-ml-solution-prototypes_document-processing_vendor_2_fake_invoices_vendor_2_invoice_10.pdf`. What would you like to do next? I can extract data, classify the document, or compare it to a vendor template.\n'}],
  'role': 'model'},
 'usage_metadata': {'candidates_token_count': 90,
  'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 90}],
  'prompt_token_count': 3753,
  'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 3753}],
  'total_token_count': 3843,
  'traffic_type': 'ON_DEMAND'},
 'invocation_id': 'e-38cbcc96-0ba3-4539-b83b-a05a136c4ef5',
 'author': 'document_agent',
 'actions': {'state_delta': {},
  'artifact_delta': {},
  'requested_auth_configs': {}},
 'id': 'mrkGE7Iw',
 'timestamp': 1749316986.057519}

In [26]:
remote_app.get_session(user_id = 'u_123', session_id = remote_session['id'])

{'id': '4622554141112139776',
 'state': {},
 'lastUpdateTime': 1749316987.03254,
 'events': [{'errorMessage': None,
   'invocationId': 'e-189f42c5-c34d-4d2a-b44e-136233d8f2cf',
   'partial': None,
   'branch': None,
   'id': '5412177212692496384',
   'turnComplete': None,
   'usageMetadata': None,
   'content': {'role': 'user',
    'parts': [{'fileData': None,
      'thought': None,
      'codeExecutionResult': None,
      'inlineData': None,
      'text': 'What do you do?',
      'videoMetadata': None,
      'functionResponse': None,
      'thoughtSignature': None,
      'executableCode': None,
      'functionCall': None}]},
   'groundingMetadata': None,
   'customMetadata': None,
   'longRunningToolIds': None,
   'errorCode': None,
   'timestamp': 1749316982.504812,
   'author': 'user',
   'interrupted': None,
   'actions': {'artifactDelta': {},
    'escalate': None,
    'transferToAgent': None,
    'skipSummarization': None,
    'requestedAuthConfigs': {},
    'stateDelta': {}}},
  

In [27]:
for event in remote_app.get_session(user_id = 'u_123', session_id = remote_session['id'])['events']:
    print(event)

{'interrupted': None, 'invocationId': 'e-189f42c5-c34d-4d2a-b44e-136233d8f2cf', 'partial': None, 'branch': None, 'id': '5412177212692496384', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'user', 'parts': [{'fileData': None, 'thought': None, 'codeExecutionResult': None, 'inlineData': None, 'text': 'What do you do?', 'videoMetadata': None, 'thoughtSignature': None, 'functionResponse': None, 'executableCode': None, 'functionCall': None}]}, 'groundingMetadata': None, 'customMetadata': None, 'longRunningToolIds': None, 'errorCode': None, 'timestamp': 1749316982.504812, 'author': 'user', 'errorMessage': None, 'actions': {'artifactDelta': {}, 'escalate': None, 'skipSummarization': None, 'transferToAgent': None, 'requestedAuthConfigs': {}, 'stateDelta': {}}}
{'errorMessage': None, 'invocationId': 'e-189f42c5-c34d-4d2a-b44e-136233d8f2cf', 'partial': None, 'branch': None, 'id': '8681790542163476480', 'turnComplete': None, 'usageMetadata': None, 'content': {'role': 'model', 'p

In [28]:
remote_app.delete_session(user_id = 'u_123', session_id = remote_session['id'])

In [29]:
remote_app.list_sessions(user_id = 'u_123')

{'sessions': []}

In [30]:
delete_app = False

if delete_app and remote_app:
    task = remote_app.delete(force = True)

more
- create session with specified id